# Cybersecurity advanced
## Lab 11: CA


Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-11/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-11/lab/)

# Demo

Open site op kali